In [ ]:
import tensorflow as tf
import ray
from ray.tune.registry import register_env
from ray.rllib.algorithms import dqn, ppo

from env.hanoiV1 import *
from agent.parametric.model import model
from agent.parametric.model import action_distribution as dist

In [2]:
%load_ext autoreload
%autoreload 2

#tf.test.is_gpu_available()

if ray.is_initialized():
    ray.shutdown()
ray.init()

2023-04-07 14:31:35,214	INFO worker.py:1553 -- Started a local Ray instance.


Python version:,3.10.6
Ray version:,2.3.0


In [ ]:
n_disks = 3
env_config = {
    'n_disks': n_disks,
    'max_n_steps': 20000 
}

ENV_NAME = 'hanoi_tower'
register_env(ENV_NAME, create_env)

config = ppo.PPOConfig()
config = config.training(lr=1e-3,
                         gamma=0.9,
                         train_batch_size=512 * 2,
                         sgd_minibatch_size=256 * 1,
                         num_sgd_iter=16 * 1)
config = config.rollouts(num_rollout_workers=1,
                         num_envs_per_worker=1)
config = config.environment(HanoiTower, 
                            env_config=env_config)
#config = config.resources(num_gpus=1)
config = config.framework('tf')

model_config = {
    'n_hiddens': 128
}

config.model.update({
    'custom_model': model.MODEL_NAME,
    'custom_model_config': model_config,
    'custom_action_dist': dist.ACTION_DIST_NAME,
    'vf_share_layers': False
})

config.seed = 19
agent = config.build()

In [ ]:
#config.to_dict()
for _ in range(100):
    res = agent.train()
agent.save()

In [ ]:
env = create_env(env_config)
obs, _ = env.reset()
obss =  []
episode_reward = 0
done = False
while not done:
    action = agent.compute_single_action(obs, explore=False)
    print(obs, action)
    obss.append(obs)
    obs, reward, done, terminated, info = env.step(action)
    episode_reward += reward


In [ ]:
obs, episode_reward